# Code 9
- Random Forest
- Hyperparameter Optimization
- GridsearchCV
- Best Features
- **Feature Engineering**

## 1/ Import Libraries

In [ ]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [ ]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [ ]:
#CodeSection3
train = pd.read_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/input/train.csv')
test = pd.read_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/input/test.csv')

## 3/ Create 2 New Features


In [ ]:
#CodeSection4
all_data = [train, test]

### 3.1 Create First Feature

In [ ]:
#CodeSection5
# Total Income is Sum of Applicant Income and Co-applicant Income
<Write your code here>

### 3.2 Create Second Feature

In [ ]:
#CodeSection6
# Loan by Income is Loan Amount divided by Total Income
<Write your code here>

## 4/ Preprocessing Improvement

### 4.1/ Identify Numerical and Categorical Features

In [ ]:
#CodeSection7
# Identify all Numerical and Categorical features
numerical_features = ['LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Total_Income', 'Loan_by_Income']
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

### 4.2/ Outlier Strategy

In [ ]:
#CodeSection8

# Write a loop to do the same
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  train[num_var] = train[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)
  test[num_var] = test[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)

### 4.3/ Import Libraries
- Missing Value Imputation - SimpleImputer
- Preprocessing - StandardScaler, OrdinalEncoder
- Pipeline - make_pipeline, make_column_transformer
- Model - Decision Tree

In [ ]:
#CodeSection9

# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Import StandardScaler
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Make and Compose Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Import Decision tree
from sklearn.ensemble import RandomForestClassifier

### 4.4/ Build Pipeline

In [ ]:
#CodeSection10
# Create Preprocessor Pipeline
preprocessor = make_column_transformer(
    
    (make_pipeline(
    SimpleImputer(strategy = 'median'), 
    StandardScaler()), numerical_features),
    
    (make_pipeline(
    SimpleImputer(strategy = 'most_frequent'),
    OrdinalEncoder(categories = 'auto')), categorical_features),
)

### 4.5/ Divide Data into X and y

In [ ]:
#CodeSection11
X = train.drop(['Loan_Status','Loan_ID','ApplicantIncome', 'CoapplicantIncome'], axis =  1)
y = train['Loan_Status']

### 4.6/ Create Train and Validation Data

In [ ]:
#CodeSection12
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, random_state = 5) # also see stratify

## 5/ Build Model and Fit

In [ ]:
#CodeSection13
# Create Parameter Grid
parameter = {'randomforestclassifier__n_estimators' : (22, 23)
              , 'randomforestclassifier__max_depth' : (2,3,4,5,6,7)
              , 'randomforestclassifier__criterion' : ('gini', 'entropy')
              , 'randomforestclassifier__max_features' : ('auto', 'sqrt', 'log2')
}

In [ ]:
#CodeSection14
# Create Model Pipeline and Initiate Model
# Change max_depth to find which one gives the best accuracy
model = make_pipeline(preprocessor, RandomForestClassifier())

In [ ]:
#CodeSection15
# Instead of Fit we so Grid Search
from sklearn.model_selection import GridSearchCV
model_search = GridSearchCV(model, param_grid = parameter, verbose = True, n_jobs = -1)

In [ ]:
#CodeSection16
# Fit Gridsearch
model_search.fit(X_train,y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   17.6s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
               

In [ ]:
#CodeSection17
# Get best estimator from the Girdsearch
model_search.best_estimator_.named_steps.randomforestclassifier

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

## 6/ Check Best Features

In [ ]:
#CodeSection18
# Get Feature Importance Score
feat_imp = model_search.best_estimator_.named_steps.randomforestclassifier.feature_importances_
feat_imp

array([0.10543477, 0.02098978, 0.49160727, 0.08526807, 0.1707341 ,
       0.01333525, 0.01154245, 0.02960476, 0.02221472, 0.01145095,
       0.03781787])

In [ ]:
#CodeSection19
# Convert to Series with Feature Names
imp_feat=pd.Series(feat_imp,index=X_train.columns.tolist())

In [ ]:
#CodeSection20
# List 
imp_feat.sort_values(ascending=False) # You also plot the same

Dependents          0.491607
Self_Employed       0.170734
Gender              0.105435
Education           0.085268
Loan_by_Income      0.037818
Credit_History      0.029605
Property_Area       0.022215
Married             0.020990
LoanAmount          0.013335
Loan_Amount_Term    0.011542
Total_Income        0.011451
dtype: float64

## 6/ Check Accuracy of Model on Train Data

In [ ]:
#CodeSection21
# Predict on Train Data
y_train_pred = model_search.predict(X_train)
y_val_pred = model_search.predict(X_val)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [ ]:
#CodeSection22
# Import metrics library
from sklearn.metrics import accuracy_score

In [ ]:
#CodeSection23
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_train, y_train_pred):0.1%}")
print(f" Validation Accuracy : {accuracy_score(y_val, y_val_pred):0.1%}")

 Train Accuracy : 85.3%
 Validation Accuracy : 81.3%


## 7/ Predict and Submission

### Predict on "Test Data"

In [ ]:
#CodeSection24
# Get all the X Variables from the Test Dataset
X_test = test.drop(['Loan_ID'], axis =  1)

# Predict on X_test Data ("X_test_prep")
X_test_prep = model_search.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


### Create Submission File

In [ ]:
#CodeSection25
submission = pd.DataFrame({
    'Loan_ID' : test['Loan_ID'],
    'Loan_Status' : X_test_prep
})

### Export Submission File

In [ ]:
#CodeSection26
submission.to_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/output/O9_Feature_Engineering.csv', index = False)

In [ ]:
# LB Accuracy : 0.7916 (Validation Accuracy : 81.3%)